In [38]:
import pymongo
from pymongo import MongoClient
import requests
from requests import ReadTimeout
from pprint import pprint
from mxnet.image import imread
from mxnet import nd
import numpy as np
from io import BytesIO
from PIL import Image
import asyncio

In [3]:
host =  "api/v1/"

In [46]:
try:
    ret = requests.get(host + "manage/models/list",timeout=1)
    print("Mongo table list:\n",ret.json())
except ReadTimeout:
    pprint("list table time out")
try:
    ret = requests.get(host + "manage/models/loaded")
    print("Faiss index list:\n",ret.json())
except ReadTimeout:
    print("list index time out")

Mongo table list:
 ['jmd_test_xb', 'word_dict', 'wangyiwen_test', 'image_faiss_dual_taobao', 'test_w2v2', 'test_w2v3', 'w2v_test_xb2', 'car_sim', 'wordvector_index', 'tutorial_test', 'image_metric_taobao128', 'sku_img_embedding', 'dengjuntest', 'test_w2v', 'test_w2v1', 'image_faiss_dual', 'w2v_test_xb', 'nnparamtest', 'image_faiss_dual_tmall', 'image_nn_prod', 'image_faiss_prod', 'w2v_total', 'car_intro_embedding', 'image_faiss_prod2', 'image_nn_test', 'w2v_formal']
Faiss index list:
 ['jmd_test_xb', 'wangyiwen_test', 'image_faiss_dual_taobao', 'test_w2v3', 'w2v_test_xb2', 'car_sim', 'wordvector_index', 'tutorial_test', 'sku_img_embedding', 'test_w2v', 'image_faiss_dual', 'w2v_test_xb', 'nnparamtest', 'image_faiss_dual_tmall', 'image_nn_prod', 'image_faiss_prod', 'w2v_total', 'car_intro_embedding', 'image_faiss_prod2', 'image_nn_test']


In [96]:
index_name = 'image_metric_taobao128'
status = requests.post(host + 'model/{0}/'.format(index_name) +"status").json()['info']

In [98]:
pprint(status)


{'config': {'compression': {'m': 64},
            'dim': 128,
            'history': [{'data_size': 0,
                         'operation': 'Initialized',
                         'time': '2018-08-09 15:21:17'}],
            'index_name': 'image_metric_taobao128',
            'metric': 'l2'},
 'faiss_index': {'ntotal': 0},
 'saved': True,
 'table': {'data_size': 28259}}


In [99]:
status = requests.post(host + 'model/{0}/'.format(index_name) +"build").json()['info']

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [100]:
status

{'config': {'index_name': 'image_metric_taobao128',
  'metric': 'l2',
  'compression': {'m': 64},
  'history': [{'time': '2018-08-09 15:21:17',
    'operation': 'Initialized',
    'data_size': 0}],
  'dim': 128},
 'table': {'data_size': 28259},
 'faiss_index': {'ntotal': 0},
 'saved': True}

In [8]:
index_name = 'image_nn_prod'
status = requests.post(host + 'model/{0}/'.format(index_name) +"status").json()['info']

In [9]:
status

{'config': {'index_name': 'image_nn_prod',
  'training_size': 262144,
  'metric': 'l2',
  'history': [{'time': '2018-01-12 14:57:06',
    'operation': 'Training',
    'data_size': 301317},
   {'time': '2018-01-12 15:15:12',
    'operation': 'Trained',
    'data_size': 301317},
   {'time': '2018-01-16 18:05:39', 'operation': 'Saved', 'data_size': 301317},
   {'time': '2018-01-17 21:47:06', 'operation': 'Saved', 'data_size': 301317},
   {'time': '2018-01-22 09:59:20', 'operation': 'Saved', 'data_size': 301317}],
  'dim': 4096,
  'use_gpu': False,
  'nprobe': 20,
  'nlist': 65536,
  'compression': {'M_mi': 2, 'nbit_mi': 8, 'm': 1024, 'nbit': 8}},
 'table': {'data_size': 301317},
 'faiss_index': {'nprobe': 5,
  'ntotal': 301317,
  'check': False,
  'id_set': 301317},
 'saved': True}

dim = 128 #向量的维度,LeNet的输出前一层是1024,AlexNet或者VggNet的全连接层多为2048
m=64 #参考选择列表[{"dim":2048,"m":256},{"dim":1024","m":128},{"dim":2048,"m":128},{"dim":1024,"m":64}]
model = 'image_metric_taobao128'
nnindex = {"dim":dim,'metric':'cosine','compression':{'m':m},'index_name':model}


path = 'model/'+model+'/'
ret = requests.post(host + path +"create",json={'nnindex':nnindex})
print("create index result",ret.json())

In [60]:
from pymongo import MongoClient
mongdb={}
#mongodb://.mongodb.rds.aliyuncs.com:3717,dds-bp10da4305cf39f41.mongodb.rds.aliyuncs.com:3717/admin?replicaSet=mgset-5028117
mongdb['host']='dds-bp10da4305cf39f42.mongodb.rds.aliyuncs.com'
mongdb['port']=3717
client=MongoClient(host=mongdb['host'],port=mongdb['port'])
dev=client.get_database('dev')
dev.authenticate(name='',password='123')

True

In [12]:
collection_list = [name for name in dev.collection_names() if not name.endswith('config')]

c:\program files\python36\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: collection_names is deprecated. Use list_collection_names instead.
  """Entry point for launching an IPython kernel.


In [13]:
nn_prod = dev.get_collection('image_nn_prod')
print(nn_prod.count())

301317


c:\program files\python36\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  


In [95]:
image_nn = dev.get_collection('image_metric_taobao128')
image_nn.count()

c:\program files\python36\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  


28259

In [71]:
cursor = image_nn.find()
for item in cursor:
    #pprint(item)
    #image_nn.delete_one({'_id':item['_id']})
    #break

In [64]:
cursor = image_nn.find()
for item in cursor:
    pprint(item)
    break

{'_id': 'ImageItem/instagram/f2bc9a0ab6c16e8223e9717efc90c347_0',
 '_index': 'image_metric_taobao128',
 '_source': {'_int_id': 14460861856687057,
             'authorid': '186901415',
             'authorname': 'victoriabeckham',
             'body_parsed': '[{"type": "img", "value": '
                            '"images/instagram/f2bc9a0ab6c16e8223e9717efc90c347"}]',
             'cdn_url': 'https://cdn.aidigger.com/images/instagram/f2bc9a0ab6c16e8223e9717efc90c347.jpeg',
             'commentcount': '1133',
             'headline': 'I am overwhelmed by your response to the launch of '
                         'my #VBxTarget collaboration! Thank you so much for '
                         'your patience and sorry for any frustration for '
                         'those shopping on victoriabeckham.com. Hold tight! x '
                         'VB',
             'height': '803',
             'id': None,
             'like': '100456',
             'pic_url': 'https://www.instagram.com/p

In [40]:
batch_size=5
cursor = nn_prod.find({},{'vector':0},batch_size=batch_size)

In [39]:
@asyncio.coroutine
def download(url,fobj):
    connect = asyncio.open_connection(url,80)
    reader,writer = yield from connect
    header = 'GET / HTTP/1.0\r\nHost: %s\r\n\r\n' % host
    writer.write(header.encode('utf-8'))
    yield from writer.drain()
    content = yield from reader.content
    fobj.write(content)
    writer.close()

In [43]:
#loop = asyncio.get_event_loop()
loop.close()

RuntimeError: Cannot close a running event loop

In [41]:
item_list = []
url_list = []

for item in cursor:
    if len(url_list)==5:
        #store and download and then insert
        fobject_list = [BytesIO() for _ in range(batch_size)]
        loop = asyncio.get_event_loop()
        tasks = [download(url,obj) for url,obj in zip(url_list,fobject_list)]
        loop.run_until_complete(asyncio.wait(tasks))
        loop.close()
        break
    item_list.append(item)
    url_list.append(item['_source']['cdn_url'])
    

RuntimeError: This event loop is already running

In [34]:
url_list

['https://cdn.aidigger.com/images/instagram/e2452f9daaad3ef7070adb22ee70958a.jpeg',
 'https://cdn.aidigger.com/images/instagram/bd717eaa4c351b842a497e8907b69855.jpeg',
 'https://cdn.aidigger.com/images/instagram/189a2af5d9661500b32271ca9b1865be.jpeg',
 'https://cdn.aidigger.com/images/instagram/6e70c94dd3fac214c5d7e6c061df2b2f.jpeg',
 'https://cdn.aidigger.com/images/instagram/f95f00da22a2e143e6e457b10544a120.jpeg']

In [ ]:
ret = requests.get(img_url)

In [ ]:
f= BytesIO()
f.write(ret.content)
img_io = Image.open(f)
np_img = np.asarray(img_io)
md_img = nd.array(np_img)

In [45]:

@asyncio.coroutine
def wget(host):
    print('wget %s...' % host)
    connect = asyncio.open_connection(host, 80)
    reader, writer = yield from connect
    header = 'GET / HTTP/1.0\r\nHost: %s\r\n\r\n' % host
    writer.write(header.encode('utf-8'))
    yield from writer.drain()
    while True:
        line = yield from reader.readline()
        if line == b'\r\n':
            break

        print('%s header > %s' % (host, line.decode('utf-8').rstrip()))
    # Ignore the body, close the socket
    writer.close()

loop = asyncio.get_event_loop()
tasks = [wget(host) for host in ['www.sina.com.cn', 'www.sohu.com', 'www.163.com']]
loop.run_until_complete(asyncio.wait(tasks))
loop.close()

RuntimeError: This event loop is already running